### All days of the challange:

Here's what we're going to do today:

* [Take a first look at the data](#Take-a-first-look-at-the-data)
* [See how many missing data points we have](#See-how-many-missing-data-points-we-have)
* [Figure out why the data is missing](#Figure-out-why-the-data-is-missing)
* [Drop missing values](#Drop-missing-values)
* [Filling in missing values](#Filling-in-missing-values)

Let's get started!

# Take a first look at the data
________

The first thing we'll need to do is load in the libraries and datasets we'll be using. For today, I'll be using a dataset of events that occured in American Football games for demonstration, and you'll be using a dataset of building permits issued in San Francisco.

> **Important!** Make sure you run this cell yourself or the rest of your code won't work!

In [2]:
# libraries we'll use
import pandas as pd

# read in all our data NFL nfl1-> Play by Play 2009-2017 (v4), nfl2 -> NFL Play by Play 2009-2016 (v3)

nfl1 = pd.read_csv("NFL Play by Play 2009-2017 (v4).csv")
nfl2 = pd.read_csv("NFL Play by Play 2009-2016 (v3).csv")

C:\Users\lenovo\AppData\Local\Temp\ipykernel_18092\2412057861.py:6: DtypeWarning: Columns (25,51) have mixed types. Specify dtype option on import or set low_memory=False.
  nfl1 = pd.read_csv("NFL Play by Play 2009-2017 (v4).csv")
C:\Users\lenovo\AppData\Local\Temp\ipykernel_18092\2412057861.py:7: DtypeWarning: Columns (25,51) have mixed types. Specify dtype option on import or set low_memory=False.
  nfl2 = pd.read_csv("NFL Play by Play 2009-2016 (v3).csv")


The first thing I do when I get a new dataset is take a look at some of it. This lets me see that it all read in correctly and get an idea of what's going on with the data. In this case, I'm looking to see if I see any missing values, which will be reprsented with `NaN` or `None`.

In [3]:
# look at a few rows of the nfl1_data file. I can see a handful of missing data already!
nfl1.head()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2009-09-10,2009091000,1,1,NaN,15:00,15,3600.0,0.0,TEN,...,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,2009-09-10,2009091000,1,1,1.0,14:53,15,3593.0,7.0,PIT,...,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,2009-09-10,2009091000,1,1,2.0,14:16,15,3556.0,37.0,PIT,...,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,2009-09-10,2009091000,1,1,3.0,13:35,14,3515.0,41.0,PIT,...,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2009-09-10,2009091000,1,1,4.0,13:27,14,3507.0,8.0,PIT,...,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


Yep, it looks like there's some missing values. What about in the nfl2 dataset?

In [4]:
# your turn! Look at a couple of rows from the nfl2 dataset. Do you notice any missing data?
nfl2.head(2)
# your code goes here :)

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2009-09-10,2009091000,1,1,NaN,15:00,15,3600.0,0.0,TEN,...,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,2009-09-10,2009091000,1,1,1.0,14:53,15,3593.0,7.0,PIT,...,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009


# See how many missing data points we have
___

Ok, now we know that we do have some missing values. Let's see how many we have in each column. 

In [5]:
# get the number of missing data points per column
nfl1.isna().sum()

# look at the # of missing points in the first ten columns
nfl1[nfl1.columns[:10]].isna().sum()

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64

That seems like a lot! It might be helpful to see what percentage of the values in our dataset were missing to give us a better sense of the scale of this problem:

In [6]:
# how many total missing values do we have?
sumOfMissingValues1 = nfl1.isna().sum().sum()

# percent of data that is missing
sumOfMissingValues1 / (len(nfl1) * len(nfl1.columns)) * 100

27.66722370547874

Wow, almost a quarter of the cells in this dataset are empty! In the next step, we're going to take a closer look at some of the columns with missing values and try to figure out what might be going on with them.

In [7]:
# your turn! Find out what percent of the nfl2_ dataset is missing
sumOfMissingValues2 = nfl2.isna().sum().sum()

# percent of data that is missing
sumOfMissingValues2 / (len(nfl2) * len(nfl2.columns)) * 100

27.652267428200588

# Figure out why the data is missing
____
 
This is the point at which we get into the part of data science that I like to call "data intution", by which I mean "really looking at your data and trying to figure out why it is the way it is and how that will affect your analysis". It can be a frustrating part of data science, especially if you're newer to the field and don't have a lot of experience. For dealing with missing values, you'll need to use your intution to figure out why the value is missing. One of the most important question you can ask yourself to help figure this out is this:

> **Is this value missing becuase it wasn't recorded or becuase it dosen't exist?**

If a value is missing becuase it doens't exist (like the height of the oldest child of someone who doesn't have any children) then it doesn't make sense to try and guess what it might be. These values you probalby do want to keep as NaN. On the other hand, if a value is missing becuase it wasn't recorded, then you can try to guess what it might have been based on the other values in that column and row. (This is called "imputation" and we'll learn how to do it next! :)

Let's work through an example. Looking at the number of missing values in the nfl_data dataframe, I notice that the column `TimesSec` has a lot of missing values in it: 

In [8]:
# look at the # of missing points in the first ten columns
nfl1.iloc[:,:10].isna().sum()

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64

By looking at [the documentation](https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016), I can see that this column has information on the number of seconds left in the game when the play was made. This means that these values are probably missing because they were not recorded, rather than because they don't exist. So, it would make sense for us to try and guess what they should be rather than just leaving them as NA's.

On the other hand, there are other fields, like `PenalizedTeam` that also have lot of missing fields. In this case, though, the field is missing because if there was no penalty then it doesn't make sense to say *which* team was penalized. For this column, it would make more sense to either leave it empty or to add a third value like "neither" and use that to replace the NA's.

> **Tip:** This is a great place to read over the dataset documentation if you haven't already! If you're working with a dataset that you've gotten from another person, you can also try reaching out to them to get more information.

If you're doing very careful data analysis, this is the point at which you'd look at each column individually to figure out the best strategy for filling those missing values. For the rest of this notebook, we'll cover some "quick and dirty" techniques that can help you with missing values but will probably also end up removing some useful information or adding some noise to your data.

## Your turn!

* Look at the columns `Street Number Suffix` and `Zipcode` from the `sf_permits` datasets. Both of these contain missing values. Which, if either, of these are missing because they don't exist? Which, if either, are missing because they weren't recorded?

# Drop missing values
___

If you're in a hurry or don't have a reason to figure out why your values are missing, one option you have is to just remove any rows or columns that contain missing values. (Note: I don't generally recommend this approch for important projects! It's usually worth it to take the time to go through your data and really look at all the columns with missing values one-by-one to really get to know your dataset.)  

If you're sure you want to drop rows with missing values, pandas does have a handy function, `dropna()` to help you do this. Let's try it out on our NFL dataset!

In [9]:
# remove all the rows that contain a missing value
nfl1.dropna()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season


Oh dear, it looks like that's removed all our data! 😱 This is because every row in our dataset had at least one missing value. We might have better luck removing all the *columns* that have at least one missing value instead.

In [10]:
# remove all columns with at least one missing value
nfl1.dropna(axis =1)

,Date,GameID,Drive,qtr,TimeUnder,ydstogo,ydsnet,PlayAttempted,Yards.Gained,sp,...,AwayTeam,Timeout_Indicator,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,ExPoint_Prob,TwoPoint_Prob,Season
0,2009-09-10,2009091000,1,1,15,0,0,1,39,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
1,2009-09-10,2009091000,1,1,15,10,5,1,5,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
2,2009-09-10,2009091000,1,1,15,5,2,1,-3,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
3,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
4,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407683,2017-12-31,2017123101,29,4,1,0,-4,1,0,0,...,CIN,1,0,3,0,2,0,0.0,0.0,2017
407684,2017-12-31,2017123101,29,4,1,14,-4,1,0,0,...,CIN,0,2,2,0,2,0,0.0,0.0,2017
407685,2017-12-31,2017123101,29,4,1,14,9,1,13,0,...,CIN,0,2,2,0,2,0,0.0,0.0,2017
407686,2017-12-31,2017123101,30,4,1,10,-1,1,-1,0,...,CIN,0,0,2,0,2,0,0.0,0.0,2017


In [11]:
# just how much data did we lose?
## got confused

We've lost quite a bit of data, but at this point we have successfully removed all the `NaN`'s from our data. 

In [12]:
# Your turn! Try removing all the rows from the nfl2 dataset that contain missing values. How many are left?
nfl2.dropna()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season


In [13]:
# Now try removing all the columns with empty values. Now how much of your data is left?
nfl2.dropna(axis=1)

,Date,GameID,Drive,qtr,TimeUnder,ydstogo,ydsnet,PlayAttempted,Yards.Gained,sp,...,AwayTeam,Timeout_Indicator,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,ExPoint_Prob,TwoPoint_Prob,Season
0,2009-09-10,2009091000,1,1,15,0,0,1,39,0,...,TEN,0,3,3,3,3,3,0.000000,0.0,2009
1,2009-09-10,2009091000,1,1,15,10,5,1,5,0,...,TEN,0,3,3,3,3,3,0.000000,0.0,2009
2,2009-09-10,2009091000,1,1,15,5,2,1,-3,0,...,TEN,0,3,3,3,3,3,0.000000,0.0,2009
3,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,TEN,0,3,3,3,3,3,0.000000,0.0,2009
4,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,TEN,0,3,3,3,3,3,0.000000,0.0,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362442,2017-01-01,2017010102,20,4,1,10,35,1,35,1,...,GB,0,0,0,0,0,0,0.000000,0.0,2016
362443,2017-01-01,2017010102,20,4,1,0,35,1,0,1,...,GB,0,0,0,0,0,0,0.931115,0.0,2016
362444,2017-01-01,2017010102,21,4,1,0,0,1,0,0,...,GB,0,0,0,0,0,0,0.000000,0.0,2016
362445,2017-01-01,2017010102,21,4,1,10,-1,1,-1,0,...,GB,0,0,0,0,0,0,0.000000,0.0,2016


# Filling in missing values automatically
_____

Another option is to try and fill in the missing values. For this next bit, I'm getting a small sub-section of the NFL data so that it will print well.

In [14]:
# get a small subset of the NFL dataset
nfl1_sample = nfl1.sample(frac = 0.0001)
nfl1_sample

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
338133,2016-10-30,2016103010,15,4,2.0,14:28,15,868.0,4.0,DAL,...,-1.792739,0.144239,0.855761,0.160102,0.839898,8.557614e-01,-0.015863,0.020232,-0.036095,2016
271841,2015-09-13,2015091301,3,1,2.0,03:20,4,2900.0,37.0,STL,...,NaN,0.373953,0.626047,0.296522,0.703478,3.739534e-01,-0.077432,NaN,NaN,2015
236474,2014-10-05,2014100507,1,1,3.0,14:49,15,3589.0,5.0,STL,...,-0.543867,0.564761,0.435239,0.604079,0.395921,4.352392e-01,-0.039318,-0.022317,-0.017001,2014
372035,2017-10-01,2017100102,11,2,2.0,00:06,1,1806.0,0.0,MID,...,0.000000,0.068226,0.931774,0.061051,0.938949,6.822606e-02,-0.007175,-0.007175,0.000000,2017
319701,2016-09-15,2016091500,19,4,1.0,03:29,4,209.0,28.0,BUF,...,-1.244712,0.008209,0.991791,0.007009,0.992991,8.208754e-03,-0.001200,0.001498,-0.002698,2016
11484,2009-10-11,2009101102,1,1,NaN,15:00,15,3600.0,0.0,WAS,...,NaN,0.485675,0.514325,0.483199,0.516801,4.856750e-01,-0.002476,NaN,NaN,2009
274786,2015-09-20,2015092002,7,2,1.0,11:52,12,2512.0,33.0,CIN,...,NaN,0.503438,0.496562,0.530838,0.469162,4.965616e-01,-0.027400,NaN,NaN,2015
18426,2009-11-01,2009110103,5,1,1.0,01:52,2,2812.0,13.0,SEA,...,NaN,0.746720,0.253280,0.722261,0.277739,7.467202e-01,-0.024459,NaN,NaN,2009
139250,2012-09-16,2012091608,6,2,2.0,14:18,15,2658.0,42.0,SEA,...,-2.338156,0.723486,0.276514,0.749303,0.250697,2.765137e-01,-0.025817,0.049153,-0.074970,2012
328413,2016-10-09,2016100901,20,4,3.0,08:33,9,513.0,6.0,CLE,...,0.465090,0.020503,0.979497,0.019311,0.980689,9.794972e-01,0.001192,-0.003508,0.004700,2016


We can use the Panda's fillna() function to fill in missing values in a dataframe for us. One option we have is to specify what we want the `NaN` values to be replaced with. Here, I'm saying that I would like to replace all the `NaN` values with 0.

In [15]:
# replace all NA's with 0
nfl1_sample.fillna(0)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_18092\2818551668.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  nfl1_sample.fillna(0)


,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
338133,2016-10-30,2016103010,15,4,2.0,14:28,15,868.0,4.0,DAL,...,-1.792739,0.144239,0.855761,0.160102,0.839898,8.557614e-01,-0.015863,0.020232,-0.036095,2016
271841,2015-09-13,2015091301,3,1,2.0,03:20,4,2900.0,37.0,STL,...,0.000000,0.373953,0.626047,0.296522,0.703478,3.739534e-01,-0.077432,0.000000,0.000000,2015
236474,2014-10-05,2014100507,1,1,3.0,14:49,15,3589.0,5.0,STL,...,-0.543867,0.564761,0.435239,0.604079,0.395921,4.352392e-01,-0.039318,-0.022317,-0.017001,2014
372035,2017-10-01,2017100102,11,2,2.0,00:06,1,1806.0,0.0,MID,...,0.000000,0.068226,0.931774,0.061051,0.938949,6.822606e-02,-0.007175,-0.007175,0.000000,2017
319701,2016-09-15,2016091500,19,4,1.0,03:29,4,209.0,28.0,BUF,...,-1.244712,0.008209,0.991791,0.007009,0.992991,8.208754e-03,-0.001200,0.001498,-0.002698,2016
11484,2009-10-11,2009101102,1,1,0.0,15:00,15,3600.0,0.0,WAS,...,0.000000,0.485675,0.514325,0.483199,0.516801,4.856750e-01,-0.002476,0.000000,0.000000,2009
274786,2015-09-20,2015092002,7,2,1.0,11:52,12,2512.0,33.0,CIN,...,0.000000,0.503438,0.496562,0.530838,0.469162,4.965616e-01,-0.027400,0.000000,0.000000,2015
18426,2009-11-01,2009110103,5,1,1.0,01:52,2,2812.0,13.0,SEA,...,0.000000,0.746720,0.253280,0.722261,0.277739,7.467202e-01,-0.024459,0.000000,0.000000,2009
139250,2012-09-16,2012091608,6,2,2.0,14:18,15,2658.0,42.0,SEA,...,-2.338156,0.723486,0.276514,0.749303,0.250697,2.765137e-01,-0.025817,0.049153,-0.074970,2012
328413,2016-10-09,2016100901,20,4,3.0,08:33,9,513.0,6.0,CLE,...,0.465090,0.020503,0.979497,0.019311,0.980689,9.794972e-01,0.001192,-0.003508,0.004700,2016


I could also be a bit more savvy and replace missing values with whatever value comes directly after it in the same column. (This makes a lot of sense for datasets where the observations have some sort of logical order to them.)

In [17]:
# replace all NA's the value that comes directly after it in the same column,
nfl1_sample = nfl1_sample.bfill()
nfl1_sample
# then replace all the reamining na's with 0
nfl1_sample = nfl1_sample.fillna(0)

Filling in missing values is also known as "imputation", and you can find more exercises on it [in this lesson, also linked under the "More practice!" section](https://www.kaggle.com/dansbecker/handling-missing-values). First, however, why don't you try replacing some of the missing values in the sf_permit dataset?

In [18]:
# i dont know what im doing in this file